# 판매자 전처리

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../CSV/처리/1.데이터/sellers_final_for_ml.csv")

In [ ]:

# 1. 파일 읽기
try:
    sellers_rows = pd.read_csv('../CSV/sellers/sellers_rows.csv')
    seller_address = pd.read_csv('../CSV/sellers/seller_address.csv')
except FileNotFoundError as e:
    print(f"❌ 파일을 찾을 수 없습니다. 경로를 확인해주세요: {e}")
    exit()


In [ ]:
# 2. 주소 데이터 병합 (id 기준)
# seller_address 파일에서 정제된 주소만 가져와서 병합합니다.
seller_address_processed = seller_address[['id', 'processed_address']]
merged = sellers_rows.merge(seller_address_processed, on='id', how='left')

In [ ]:
# 3. 불필요한 컬럼 삭제
cols_to_drop = [
    'business_address',              # 정제된 주소(processed_address)로 대체
    'company_name', 
    'business_registration_number', 
    'email', 
    'phone', 
    'mail_order_number', 
    'created_at'
]
# 실제 데이터에 존재하는 컬럼만 골라 삭제합니다.
merged = merged.drop(columns=[c for c in cols_to_drop if c in merged.columns])


In [ ]:
# 4. 판매자 이름 정제 (is_english_name 생성)
if 'representative_name' in merged.columns:
    print("✓ 'representative_name' 컬럼을 기준으로 이름 정제를 수행합니다.")
    # 영문(a-z, A-Z)이 포함되어 있으면 1, 아니면 0
    merged['is_english_name'] = merged['representative_name'].str.contains('[a-zA-Z]', regex=True).fillna(False).astype(int)
else:
    print("⚠️ 'representative_name' 컬럼을 찾을 수 없습니다.")
    merged['is_english_name'] = 0

In [ ]:
# 5. 컬럼 순서 정리 (보기 좋게 배치)
# 중요 정보인 id, 이름, 주소를 앞쪽으로 배치합니다.
important_cols = ['id', 'representative_name', 'is_english_name', 'processed_address']
# 실제로 존재하는 중요 컬럼들만 리스트에 담고 나머지를 붙입니다.
existing_important = [c for c in important_cols if c in merged.columns]
other_cols = [c for c in merged.columns if c not in existing_important]
merged = merged[existing_important + other_cols]

In [ ]:
# 6. 최종 파일 저장 (맥 엑셀 깨짐 방지용 utf-8-sig)
output_path = '/Users/dobby/Desktop/team_project/coupang/CSV/sellers_final_cleaned.csv'
merged.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n✨ 전처리 파이프라인 완료!")
print(f"✓ 최종 저장 경로: {output_path}")
print(f"✓ 현재 컬럼 구성: {merged.columns.tolist()}")
print(f"✓ 총 데이터 개수: {len(merged)} 행")

In [ ]:

# 1. 파일 읽기
try:
    sellers = pd.read_csv('../CSV/sellers/sellers_rows.csv')
    address = pd.read_csv('../CSV/sellers/seller_address.csv')
except FileNotFoundError as e:
    print(f"❌ 파일을 찾을 수 없습니다. 경로를 확인해주세요: {e}")
    exit()

In [ ]:
# 2. 주소 병합
merged = sellers.merge(address[['id', 'processed_address']], on='id', how='left')

In [ ]:
 #3. 해외 사업자 여부 생성 (이게 되어야 이름 정제가 정확해짐)
korea_keywords = ['서울', '경기', '인천', '부산', '대구', '광주', '대전', '울산', '세종', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주']
merged['is_cross_border'] = merged['processed_address'].apply(
    lambda x: 0 if any(k in str(x) for k in korea_keywords) else 1
)

In [ ]:
 #3. 해외 사업자 여부 생성 (이게 되어야 이름 정제가 정확해짐)
korea_keywords = ['서울', '경기', '인천', '부산', '대구', '광주', '대전', '울산', '세종', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주']
merged['is_cross_border'] = merged['processed_address'].apply(
    lambda x: 0 if any(k in str(x) for k in korea_keywords) else 1
)

In [ ]:
# 4. 판매자 이름 정제 (이름에 영문 포함 + 해외 주소인 경우만 1)
if 'representative_name' in merged.columns:
    merged['is_english_name'] = merged.apply(
        lambda x: 1 if (pd.Series(x['representative_name']).str.contains('[a-zA-Z]', regex=True).iloc[0] 
                       and x['is_cross_border'] == 1) else 0, axis=1
    )


In [ ]:
# 5. [중요] 주소 고유 종류(nunique) 확인
unique_address_count = merged['processed_address'].nunique()

print("\n" + "="*50)
print(f"📊 주소 종류 분석 결과")
print(f"👉 중복을 제외한 주소(구/도시) 종류: {unique_address_count}개")
print("="*50 + "\n")

In [ ]:
# 6. 불필요한 텍스트 컬럼 삭제 (모델링용 전단계)
# 원본 이름과 주소 텍스트를 제거한 '숫자 중심' 데이터프레임 생성
cols_to_drop = ['representative_name', 'business_address', 'company_name', 
                'business_registration_number', 'email', 'phone', 
                'mail_order_number', 'created_at']
existing_drops = [c for c in cols_to_drop if c in merged.columns]
ml_ready_df = merged.drop(columns=existing_drops)

In [ ]:
# 7. 최종 저장
output_path = '/Users/dobby/Desktop/team_project/coupang/CSV/sellers_ml_ready.csv'
ml_ready_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ 처리가 완료되었습니다!")
print(f"✅ 저장 위치: {output_path}")
print(f"💡 위에서 나온 '주소 종류 개수'를 저에게 알려주세요! 원-핫 인코딩 여부를 정해드릴게요.")